# Importing Libraries

In [1]:
#import libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
from folium import plugins
import json

In [2]:
#ensure charts created don't need to be called 
%matplotlib inline

# Importing data 

In [3]:
#importing '.json' file for denver neighborhoods
neighborhood_geo = r'/Users/scarlettgrim/Python Projects/04-2024 Denver Airbnb Analysis/Data/Original Data/Denver_neighborhoods.geojson'

In [4]:
#making it so that i can look at the JSON file contents
f = open(r'/Users/scarlettgrim/Python Projects/04-2024 Denver Airbnb Analysis/Data/Original Data/Denver_neighborhoods.geojson')

#returning JSON object as data dictionary
data = json.load(f)

#iterating through JSON list
for i in data['features']:
    print(i)

{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-104.998122, 39.791022], [-104.998176, 39.782565], [-104.998357, 39.782611], [-104.999343, 39.782875], [-105.000292, 39.783135], [-105.001416, 39.783473], [-105.002047, 39.783599], [-105.002421, 39.783648], [-105.002838, 39.783673], [-105.002951, 39.783667], [-105.003172, 39.783654], [-105.00351, 39.783627], [-105.004259, 39.78353], [-105.005039, 39.78342], [-105.005693, 39.783316], [-105.00597, 39.78327], [-105.006205, 39.783267], [-105.006277, 39.783267], [-105.006635, 39.783264], [-105.006694, 39.783264], [-105.007044, 39.78326], [-105.007681, 39.783334], [-105.00838, 39.783403], [-105.008874, 39.78346], [-105.009732, 39.783539], [-105.009894, 39.78355], [-105.010517, 39.783594], [-105.011082, 39.783611], [-105.014818, 39.783688], [-105.016229, 39.783717], [-105.018281, 39.783709], [-105.019266, 39.783706], [-105.019481, 39.783707], [-105.019763, 39.783718], [-105.0201, 39.783747], [-105.020333, 39.783782], 

In [5]:
#creating a path variable for orginal data
path = r'/Users/scarlettgrim/Python Projects/04-2024 Denver Airbnb Analysis'

In [6]:
#checking path variable output 
path

'/Users/scarlettgrim/Python Projects/04-2024 Denver Airbnb Analysis'

In [7]:
#using path variable to import denver_airbnb.csv
df_airbnb = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'listings_checked_2.csv'), index_col = False)

# Data Wrangling

In [8]:
df_airbnb.head()

Unnamed: 0                  id    host_id  host_since  host_is_superhost  \
0           0  607435380788232654  430149575  2021-11-02              False   
1           1  545714833502855511  169214047  2018-01-22              False   
2           2            52429527  107279139  2016-12-14               True   
3           3  632494576047532593  416194740  2021-07-31               True   
4           4  687768125161080215  133612752  2017-06-05              False   

          location   latitude  longitude         property_type  \
0  North Park Hill  39.760390 -104.92968  Private room in home   
1             Hale  39.727850 -104.93783    Entire rental unit   
2      Five Points  39.758520 -104.98846      Entire townhouse   
3      West Colfax  39.736019 -105.05072      Entire townhouse   
4        Sunnyside  39.771430 -105.02028           Entire home   

         room_type  ...  review_scores_accuracy  review_scores_cleanliness  \
0     Private room  ...                    2.50                       2.50   
1  Entire home/apt  ...                    0.00                       0.00   
2  Entire home/apt  ...                    4.88                       4.62   
3  Entire home/apt  ...                    0.00                       0.00   
4  Entire home/apt  ...                    5.00                       5.00   

   review_scores_checkin  review_scores_communication  review_scores_location  \
0                   3.00                         3.50                    4.00   
1                   0.00                         0.00                    0.00   
2                   4.78                         4.78                    4.93   
3                   0.00                         0.00                    0.00   
4                   5.00                         5.00                    4.92   

   review_scores_value  instant_bookable  reviews_per_month  entire_home_apt  \
0                 3.00             False               0.11            False   
1                 0.00             False               0.00             True   
2                 4.59             False               2.52             True   
3                 0.00              True               0.00             True   
4                 4.92             False               0.99             True   

   Rating category  
0       Low rating  
1       No ratings  
2      High rating  
3       No ratings  
4      High rating  

[5 rows x 36 columns]

In [9]:
#deleting the new index column I don't need 
df_airbnb.drop(columns = ['Unnamed: 0'], inplace = True)

In [10]:
df_airbnb.shape

(4816, 35)

In [11]:
#creating a column that holds the average rating value by id
#creating a variable that holds the columns needed
reviews_to_average = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
                      'review_scores_checkin', 'review_scores_communication', 'review_scores_location']

#calculate the average for each location and store it in a series
average_rating = df_airbnb.groupby('id')[reviews_to_average].mean().mean(axis=1)

#using .map function to match values in the seires to the dataframe and fill the new column
df_airbnb['average_ratings'] = df_airbnb['id'].map(average_rating)


In [12]:
#checking output 
df_airbnb.head()

id    host_id  host_since  host_is_superhost  \
0  607435380788232654  430149575  2021-11-02              False   
1  545714833502855511  169214047  2018-01-22              False   
2            52429527  107279139  2016-12-14               True   
3  632494576047532593  416194740  2021-07-31               True   
4  687768125161080215  133612752  2017-06-05              False   

          location   latitude  longitude         property_type  \
0  North Park Hill  39.760390 -104.92968  Private room in home   
1             Hale  39.727850 -104.93783    Entire rental unit   
2      Five Points  39.758520 -104.98846      Entire townhouse   
3      West Colfax  39.736019 -105.05072      Entire townhouse   
4        Sunnyside  39.771430 -105.02028           Entire home   

         room_type  guest_count  ...  review_scores_cleanliness  \
0     Private room            1  ...                       2.50   
1  Entire home/apt            6  ...                       0.00   
2  Entire home/apt            8  ...                       4.62   
3  Entire home/apt            5  ...                       0.00   
4  Entire home/apt            3  ...                       5.00   

   review_scores_checkin  review_scores_communication  review_scores_location  \
0                   3.00                         3.50                    4.00   
1                   0.00                         0.00                    0.00   
2                   4.78                         4.78                    4.93   
3                   0.00                         0.00                    0.00   
4                   5.00                         5.00                    4.92   

   review_scores_value  instant_bookable  reviews_per_month  entire_home_apt  \
0                 3.00             False               0.11            False   
1                 0.00             False               0.00             True   
2                 4.59             False               2.52             True   
3                 0.00              True               0.00             True   
4                 4.92             False               0.99             True   

   Rating category  average_ratings  
0       Low rating         3.083333  
1       No ratings         0.000000  
2      High rating         4.795000  
3       No ratings         0.000000  
4      High rating         4.986667  

[5 rows x 36 columns]

In [13]:
df_airbnb.shape

(4816, 36)

In [14]:
#grouping the mean average rating by location = neighborhoods in denver
neighborhood_avg_ratings = df_airbnb.groupby('location')['average_ratings'].mean().reset_index()


In [15]:
#checking output
print(neighborhood_avg_ratings)

         location  average_ratings
0     Athmar Park         4.096636
1         Auraria         3.747500
2           Baker         4.514641
3          Barnum         3.785539
4     Barnum West         4.104103
..            ...              ...
72    West Colfax         4.096959
73  West Highland         4.333321
74       Westwood         3.501667
75       Whittier         4.649709
76        Windsor         3.230729

[77 rows x 2 columns]


# Data Cleaning

In [16]:
#undo displayed column limit 
pd.set_option('display.max_columns',None)

In [17]:
#creating subset with only location and average_ratings
columns_to_keep = ['location', 'average_ratings']
df_data_to_plot = df_airbnb[columns_to_keep]

In [22]:
neighborhood_avg = df_data_to_plot.groupby('location')['average_ratings'].mean().reset_index()

In [23]:
neighborhood_avg.head()

location  average_ratings
0  Athmar Park         4.096636
1      Auraria         3.747500
2        Baker         4.514641
3       Barnum         3.785539
4  Barnum West         4.104103

In [18]:
#checking output 
df_data_to_plot.shape

(4816, 2)

In [19]:
#finding missing values 
df_data_to_plot.isnull().sum()

location           0
average_ratings    0
dtype: int64

In [20]:
#checking for duplicates
dups = df_data_to_plot.duplicated()

In [21]:
#no duplicates
dups.shape

(4816,)

# Plotting the Choropleth

In [54]:
#creatinga. chroopleth map for average ratings using Denver_neighborhoods.geojson

rev_map = folium.Map(location = [39.75,-104.75], zoom_start = (10))

folium.Choropleth(
    geo_data = neighborhood_geo,
    data = neighborhood_avg,
    columns = ['location', 'average_ratings'], 
    key_on = 'feature.properties.neighbourhood',
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Average Rating").add_to(rev_map)
folium.LayerControl().add_to(rev_map) 

rev_map

In [53]:
#saving map
rev_map.save('review_by_neighborhood_map.html')

In [55]:
#saving revised dataset 
df_airbnb.to_csv(os.path.join(path, 'Data','Prepared Data', 'listings_new_columns.csv'))

# Discussion

# This map gives me an understanding to where the neighborhoods are that have the highest reviews and where have the lowest reviews. Now I am curious about the difference in housing size or type of housing based on the more popular areas. I also think that some sort of labels would be useful. The map right now gives me an idea of the distribution based on average reviews around Denver. But I would like to know more about the most popular and least popular areas. So it would be interesting to look at neighborhoods by average reveiw along with average bedrooms, average bathrooms, average price, average guest count to get a better understanding of why certain areas are more popular than others. 